In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings
warnings.filterwarnings('ignore')
random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [22]:
def gbdt_lr_train_test(File):
    #GBDT
    start = time.clock()
    train_df, test_df = train_test_split(File, train_size = 0.75, random_state=10)
    X_train = train_df.drop(train_df.columns[0], axis=1)
    y_train = train_df[train_df.columns[0]]
    X_test = test_df.drop(test_df.columns[0], axis=1)
    y_test = test_df[test_df.columns[0]]
    
    '''
    #-------------------------------------------------------------------------------------------------------------
    gbclf = GradientBoostingClassifier(n_estimators=20, max_depth=4, verbose=0)
    tuned_parameter = [{'n_estimators':[20,30,40,50], 'max_depth':[5, 6, 7, 8], 'max_features':[0.3,0.4,0.5]}]
    gs_clf = GridSearchCV(gbclf, tuned_parameter, cv=5, scoring='roc_auc')
    gs_clf.fit(X_train, y_train)
    print('best parameters set found: ')
    print(gs_clf.best_params_)
    
    y_pred_gbdt = gs_clf.predict_proba(X_test)[:, 1]
    gbdt_auc = roc_auc_score(y_test, y_pred_gbdt)
    print('gbdt auc: %.5f' % gbdt_auc)
    
    gbclf=GradientBoostingClassifier(**gs_clf.best_params_)
    #-------------------------------------------------------------------------------
    '''
    gbclf = GradientBoostingClassifier(n_estimators=30, max_depth = 7, verbose= 0,random_state=10)
    
    #-------------------------------------------------------------------------------
    
    gbclf.fit(X_train,y_train)

    num_leaf= 400
    
    y_pred=gbclf.apply(X_train.values).astype(int)
    print(y_pred)
    print(y_pred.shape)
    X_train_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_train_leaves[i][temp] += 1

    y_pred=gbclf.apply(X_test.values).astype(int)
    X_test_leaves = np.zeros([len(y_pred),len(y_pred[0]) * num_leaf],dtype=np.int64)
    for i in range(0,len(y_pred)):
        temp = np.arange(len(y_pred[0])) * num_leaf - 1 + np.array(y_pred[i])
        X_test_leaves[i][temp] += 1
    
    
    #GBDT+LR
    lr = LogisticRegression(penalty='l1', C=0.2)
    lr.fit(X_train_leaves, y_train)
    y_pred_gbdtlr1 = lr.predict_proba(X_test_leaves)[:,1]
    gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
    print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)
    
    lr = LogisticRegression(n_jobs=-1)
    X_train_ext = hstack([X_train_leaves, X_train])
    lr.fit(X_train_ext, y_train)
    X_test_ext = hstack([X_test_leaves, X_test])
    y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
    gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
    print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)
    f_time =time.clock()-start
    print('GBDT+LR time taken: %.2f'% f_time)

In [10]:
example = pd.read_csv('example2.csv')

In [13]:
gbdt_lr_train_test(example)

gbdt+lr auc 1: 0.93901
gbdt+lr auc 2: 0.91753
GBDT+LR time taken: 8.82


In [23]:
gbdt_lr_train_test(example)

[[[ 72]
  [ 68]
  [ 72]
  ..., 
  [117]
  [ 69]
  [147]]

 [[ 72]
  [ 68]
  [ 72]
  ..., 
  [117]
  [ 20]
  [147]]

 [[167]
  [165]
  [177]
  ..., 
  [200]
  [168]
  [198]]

 ..., 
 [[149]
  [146]
  [158]
  ..., 
  [185]
  [119]
  [166]]

 [[103]
  [ 99]
  [113]
  ..., 
  [ 10]
  [135]
  [  8]]

 [[103]
  [100]
  [119]
  ..., 
  [ 83]
  [102]
  [ 83]]]
(6531, 30, 1)
gbdt+lr auc 1: 0.94621
gbdt+lr auc 2: 0.90313
GBDT+LR time taken: 7.15
